# Automated ML

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
# from azureml.core.dataset import Dataset
from azureml.core import Dataset, Datastore
from azureml.pipeline.steps import AutoMLStep

## Dataset

### Overview
Stellar Classification uses the spectral data of stars to categorize them into different categories. The modern stellar classification system is known as the Morgan–Keenan (MK) classification system. It uses the old HR classification system to categorize stars with their chromaticity and uses Roman numerals to categorize the star’s size.
In this Dataset, we will be using Absolute Magnitude and B-V Color Index to Identify Giants and Dwarfs.

#### Data content:
This Dataset contains several features of Stars:
1. Vmag: Visual Apparent Magnitude of the Star
2. Plx: Distance Between the Star and the Earth
3. e_Plx: Standard Error of Plx (Drop the Row if you find the e_Plx is too high!)
4. B-V: B-V color index. (A hot star has a B-V color index close to 0 or negative, while a cool star has a B-V color index close to 2.0.
5. Amag: Absolute Magnitude of the Star

#### Activity of Project:
In this capstone project we will be predicting the TargetClass given the above feature as input.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'demo-experiment'
experiment=Experiment(ws, experiment_name)

In [3]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = " spectral dataset of stars"
description_text = "spectral data of stars dataset for udacity capstone project"
if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 
if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/kalimi03/nd00333-capstone/master/Star3642_balanced.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)

df = dataset.to_pandas_dataframe().dropna()
df.pop("SpType")
df.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


## Creating or attaching a compute cluster target

In [5]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

amlcompute_cluster_name = "demo-cluster"
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 1)


Found existing cluster, use it.
Succeeded............
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## AutoML Configuration

The details of Azure ML configuration settings have been explained in the ReadMe document.

In [6]:
project_folder = './insurance-project'
automl_settings = {
    "n_cross_validations": 5,
    "experiment_timeout_minutes": 25,
    "max_concurrent_iterations": 5,
    "enable_early_stopping": True,
    "primary_metric": 'accuracy'
}
automl_config = AutoMLConfig(compute_target=compute_target,
                            task = 'classificatin',
                            training_data=dataset,
                            label_column_name ="TargetClass",
                            path = project_folder,
                            **automl_settings
                            )

In [7]:
remote_run = experiment.submit(automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on demo-cluster with default configuration
Running on remote compute: demo-cluster


Experiment,Id,Type,Status,Details Page,Docs Page
demo-experiment,AutoML_a7ae500f-ea7e-493b-9afb-3b6fb2ef9ba7,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more about high cardinality feature handling: https://aka.ms/AutomatedMLFeaturization

********************************************

## Run Details

In [8]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [21]:
# return details of the run
remote_run.wait_for_completion(show_output=True)

Experiment,Id,Type,Status,Details Page,Docs Page
demo-experiment,AutoML_a7ae500f-ea7e-493b-9afb-3b6fb2ef9ba7,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more about high cardinality feature handling: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

****************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary de

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [9]:
best_run, model = remote_run.get_output()
best_run

Package:azureml-automl-runtime, training version:1.35.1, current version:1.34.0
Package:azureml-core, training version:1.35.0.post1, current version:1.34.0
Package:azureml-dataprep, training version:2.23.2, current version:2.22.2
Package:azureml-dataprep-rslex, training version:1.21.2, current version:1.20.1
Package:azureml-dataset-runtime, training version:1.35.0, current version:1.34.0
Package:azureml-defaults, training version:1.35.0, current version:1.34.0
Package:azureml-interpret, training version:1.35.0, current version:1.34.0
Package:azureml-mlflow, training version:1.35.0, current version:1.34.0
Package:azureml-pipeline-core, training version:1.35.0, current version:1.34.0
Package:azureml-responsibleai, training version:1.35.0, current version:1.34.0
Package:azureml-telemetry, training version:1.35.0, current version:1.34.0
Package:azureml-train-automl-client, training version:1.35.0, current version:1.34.0
Package:azureml-train-automl-runtime, training version:1.35.1, current

Experiment,Id,Type,Status,Details Page,Docs Page
demo-experiment,AutoML_a7ae500f-ea7e-493b-9afb-3b6fb2ef9ba7_39,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [10]:
model

RegressionPipeline(pipeline=Pipeline(memory=None,
                                     steps=[('datatransformer',
                                             DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='regression', working_dir='/mnt/batch/ta...
), random_state=None))], verbose=False)), ('5', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('elasticnet', ElasticNet(alpha=0.001, copy_X=True, fit_intercept=True, l1_ratio=1, max_iter=1000, normalize=False, positive=False, precompute=False, random_state=None, selection='cyclic', tol=0.0001, warm_start=False))], verbose=False))], meta_learner=ElasticNetCV(alphas=None, copy_X=True, cv=None, eps=0.001, fit_intercept=True, l1_ratio=0.5, max_iter=1000, n_alphas=100, n_jobs=None, normalize=False, positive=False, precompute=

In [11]:
model.steps

[('datatransformer',
  DataTransformer(
      task='regression',
      is_onnx_compatible=False,
      enable_feature_sweeping=True,
      enable_dnn=False,
      force_text_dnn=False,
      feature_sweeping_timeout=86400,
      featurization_config=None,
      is_cross_validation=True,
      feature_sweeping_config={}
  )),
 ('stackensembleregressor',
  StackEnsembleRegressor(
      base_learners=[('1', Pipeline(
          memory=None,
          steps=[('maxabsscaler', MaxAbsScaler(
              copy=True
          )), ('xgboostregressor', XGBoostRegressor(
              random_state=0,
              n_jobs=1,
              problem_info=ProblemInfo(
                  gpu_training_param_dict={'processing_unit_type': 'cpu'}
              ),
              tree_method='auto'
          ))],
          verbose=False
      )), ('24', Pipeline(
          memory=None,
          steps=[('standardscalerwrapper', StandardScalerWrapper(
              copy=True,
              with_mean=False,
     

In [12]:
dir_name = 'infer_dir'
if dir_name not in os.listdir():
    os.mkdir(dir_name)

In [13]:
#TODO: Save the best model
import pickle
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'infer_dir/score.py')
best_run.download_file('outputs/model.pkl', 'infer_dir/model.pkl')
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'infer_dir/conda_env.yml')



# save the model to disk
#filename = 'best_model.sav'
#pickle.dump(model, open(filename, 'wb'))

# https://docs.microsoft.com/en-us/azure/machine-learning/tutorial-auto-train-models

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [14]:
# Register the model
import sklearn

from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration

model_name = best_run.properties['model_name']
run_id = best_run.id
experiment_name = best_run.experiment.name

model = Model.register(workspace=ws,
                       model_name=model_name, # Name of the registered model in your workspace.
                       model_path='infer_dir/model.pkl',  # Local file to upload and register as a model.
                       model_framework=Model.Framework.SCIKITLEARN,  # Framework used to create the model.
                       model_framework_version=sklearn.__version__,  # Version of scikit-learn used to create the model.
                       description='Auto ML model to predict TargetClass.',
                       tags={'area': 'TargetClass', 'type': 'classification'})

print('Name:', model.name)
print('Version:', model.version)

Registering model AutoMLa7ae500fe39
Name: AutoMLa7ae500fe39
Version: 1


In [15]:
# create inference configuration
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

myenv = Environment.from_conda_specification(name="myenv", file_path="infer_dir/conda_env.yml")
inference_config = InferenceConfig(entry_script="infer_dir/score.py", environment=myenv)

# display the environment file
with open('infer_dir/conda_env.yml', 'r') as file:
    env_file = file.read()
    print(env_file)

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
  - azureml-train-automl-runtime==1.35.1
  - inference-schema
  - azureml-interpret==1.35.0
  - azureml-defaults==1.35.0
- numpy>=1.16.0,<1.19.0
- pandas==0.25.1
- scikit-learn==0.22.1
- py-xgboost<=0.90
- fbprophet==0.5
- holidays==0.9.11
- psutil>=5.2.2,<6.0.0
channels:
- anaconda
- conda-forge



TODO: In the cell below, send a request to the web service you deployed to test it.

In [16]:
# Model Deployment
from azureml.core.webservice import AciWebservice

# define deployment configuration
aci_deployment_config = AciWebservice.deploy_configuration(cpu_cores=1,
                                                           memory_gb=1,
                                                           tags={'area': "TargetClass", 'type': "classification"},
                                                           description="Predict TargetClass using classification model",
                                                           enable_app_insights=True)

# deploy model as webservice using Azure Container Instance(ACI)
aci_service_name = "aci-medical-charges-deploy"

aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aci_deployment_config, overwrite=True)
aci_service.wait_for_deployment(show_output=True)

print(aci_service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-12-04 10:35:28+00:00 Creating Container Registry if not exists..
2021-12-04 10:45:29+00:00 Registering the environment..
2021-12-04 10:45:30+00:00 Building image..
2021-12-04 10:57:02+00:00 Generating deployment configuration.
2021-12-04 10:57:03+00:00 Submitting deployment to compute..
2021-12-04 10:57:06+00:00 Checking the status of deployment aci-medical-charges-deploy..
2021-12-04 11:01:27+00:00 Checking the status of inference endpoint aci-medical-charges-deploy.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [17]:
# get the active api endpoint for scoring
print(aci_service.scoring_uri)

http://2645b839-4bf0-4aa4-bdca-6f175f1bd29a.southcentralus.azurecontainer.io/score


In [18]:
# run script to score the 4 observations below in the json payloader
import json
import requests

scoring_uri = 'http://2645b839-4bf0-4aa4-bdca-6f175f1bd29a.southcentralus.azurecontainer.io/score'
headers = {'Content-Type':'application/json'}

test_sample = json.dumps({
    "data": [
            [ 5.99, 13.73, 0.58, 1.318, 16.678352 ],
            [ 8.6, 5.09, 1.37, 0.448, 17.133589 ]
        ]
    })

response = requests.post(scoring_uri, data=test_sample, headers=headers)
print("Results:", response.json())

Results: {"result": [19684.820170258816, 2808.3080752085543]}


TODO: In the cell below, print the logs of the web service and delete the service

In [19]:
# print the log of the webservice
print(aci_service.get_logs())

2021-12-04T11:01:12,829366200+00:00 - rsyslog/run 
2021-12-04T11:01:12,828331900+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2021-12-04T11:01:12,868173100+00:00 - iot-server/run 
2021-12-04T11:01:12,906573000+00:00 - nginx/run 
rsyslogd: /azureml-envs/azureml_3c16aae97c7a45ee5cfd748a3ac40d13/lib/libuuid.so.1: no version information available (required by rsyslogd)
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-12-04T11:01:13,476825800+00:00 - iot-server/finish 1 0
2021-12-04T11:01:13,483742000+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (72)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 100
SPARK_HOME not set. Skipping PySpark Initialization.
Generating new fontManager, this may take some time...
Initializing logger
2021-12-04 11:01:16,095 | root | INFO | Starting up app insights client
logging socket was

In [20]:
# delete the web service
aci_service.delete()
model.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
